In [1]:
import torch
from project_helper import TweetData
import numpy as np
import pandas as pd
import os

In [2]:
tweet_data = TweetData()

In [3]:
daily_tweets = tweet_data.clean_tweets #[pd.to_datetime(tweet_data.clean_tweets.after4_date)
                                       #<= pd.to_datetime(daily_df.index[-1])]
daily_tweets.after4_date = pd.to_datetime(daily_tweets.after4_date)

In [4]:
daily_tweets.head()

,tweets,timestamp,after4_date
timestamp,,,
2019-11-17 19:57:12-06:00,tell jennifer williams whoever that is to read...,2019-11-17 19:57:12-06:00,2019-11-18
2019-11-17 19:56:02-06:00,,2019-11-17 19:56:02-06:00,2019-11-18
2019-11-17 19:49:47-06:00,paul krugman of has been wrong about me from t...,2019-11-17 19:49:47-06:00,2019-11-18
2019-11-17 19:47:32-06:00,schiff is a corrupt politician,2019-11-17 19:47:32-06:00,2019-11-18
2019-11-17 19:30:09-06:00,blew the nasty amp obnoxious chris wallace wil...,2019-11-17 19:30:09-06:00,2019-11-18


In [5]:
model = torch.load('models/trump_rnn_1911.net')

In [6]:
embeddings = model.in_embed.weight.to('cpu').data.numpy()

In [7]:
embeddings.shape

(4574, 100)

In [8]:
arches = os.listdir('models')
arches[0:5]

['trump_rnn_1702.net',
 'trump_rnn_1703.net',
 'trump_rnn_1908.net',
 'trump_rnn_1707.net',
 'trump_rnn_1712.net']

In [9]:
embedding_list = []
for arch in arches:
    model = torch.load('models/{}'.format(arch))
    embedding_list.append(model.in_embed.weight.to('cpu').data.numpy())
    

In [10]:
embedding_list[0].shape

(2576, 100)

In [11]:
data = TweetData('data/opt/trump_archive_db_1911.csv')
words = data.words
vocab_to_int, int_to_vocab = data.vocab_to_int, data.int_to_vocab
int_words = data.int_words

In [12]:
embedding_list[-1]

array([[-0.07983071,  0.13481925, -0.3339518 , ...,  0.07891949,
        -0.03683575, -0.06613905],
       [-0.4210904 ,  0.02706584, -0.1898706 , ..., -0.17446454,
        -0.08358873, -0.03138035],
       [-0.22094908, -0.20108747, -0.04167196, ...,  0.11216594,
         0.00401473,  0.15296404],
       ...,
       [-0.38425526, -0.3069386 ,  0.7174769 , ..., -0.4966805 ,
        -0.22396143, -0.90884376],
       [-1.4828042 ,  0.38311642,  0.23013367, ...,  0.69838697,
         0.10621499,  0.49422497],
       [-0.72725123,  0.19940047, -0.78007627, ...,  0.00616149,
         0.37083137, -0.43986282]], dtype=float32)

In [13]:
daily_tweets.after4_date

timestamp
2019-11-17 19:57:12-06:00   2019-11-18
2019-11-17 19:56:02-06:00   2019-11-18
2019-11-17 19:49:47-06:00   2019-11-18
2019-11-17 19:47:32-06:00   2019-11-18
2019-11-17 19:30:09-06:00   2019-11-18
                               ...    
2009-05-12 14:07:28-05:00   2009-05-12
2009-05-08 20:40:15-05:00   2009-05-09
2009-05-08 13:38:08-05:00   2009-05-08
2009-05-05 01:00:10-05:00   2009-05-05
2009-05-04 18:54:02-05:00   2009-05-05
Name: after4_date, Length: 28813, dtype: datetime64[ns]

In [25]:
tweet_embeddings = {}
dtws = daily_tweets.tweets[daily_tweets.after4_date >= pd.to_datetime('1-1-2017')]
tweet_embeddings_np = np.zeros([dtws.shape[0],100])

for i, tweet in enumerate(dtws):
    embed_sum = np.zeros(100) 
    tw = tweet.split()
    wd_count = 0
    for word in tw:
        try:
            if vocab_to_int[word] >= 25:
                embed_sum += embedding_list[-1][vocab_to_int[word]]
                wd_count += 1

        except:
            pass
    if wd_count > 0:    
        tweet_embeddings[i] = embed_sum/wd_count
        tweet_embeddings_np[i,:] = embed_sum/wd_count
    else:
        tweet_embeddings[i] = embed_sum
        tweet_embeddings_np[i,:] = embed_sum
        
            
        

In [27]:
tweet_embeddings_np.shape

(9582, 100)

In [26]:
pd.DataFrame(tweet_embeddings_np).to_csv('tweet_embeddings.csv')

In [23]:
tw = 1000

embedding = model.in_embed
embed_vectors = embedding.weight
magnitudes = embed_vectors.pow(2).sum(dim=1).sqrt().unsqueeze(0)
tweet_after = daily_tweets.tweets[daily_tweets.after4_date >= pd.to_datetime('1-1-2017')]
valid_vector = torch.FloatTensor(tweet_embeddings[tw])
wds = (torch.matmul(valid_vector, embed_vectors.t()) / torch.FloatTensor(magnitudes)).topk(10)
for wd in list(wds[1][0].numpy()):
    if wd> 25 :
        print(int_to_vocab[wd])
tweet_after[tw]

collusion
democrats
friday
president
up
do
has


'democrats wrote to the ukrainian government in may urging it to continue investigations into president donald trumps alleged collusion with russia in the presidential campaign collusion later found not to exist '

In [ ]:
wds

In [24]:
tw = 1467

embedding = model.in_embed
embed_vectors = torch.FloatTensor(tweet_embeddings_np)
magnitudes = embed_vectors.pow(2).sum(dim=1).sqrt().unsqueeze(0)
tweet_after = daily_tweets.tweets[daily_tweets.after4_date >= pd.to_datetime('1-1-2017')]
valid_vector = torch.FloatTensor(tweet_embeddings[tw])
wds = (torch.matmul(valid_vector, embed_vectors.t()) / torch.FloatTensor(magnitudes))
wds[wds != wds] = 0
print(wds.topk(10))
for i in wds.topk(10).indices.numpy():
    for tweet_range in i:
        print(tweet_after[tweet_range])
        print('')


torch.return_types.topk(
values=tensor([[40.2167, 35.8954, 35.8427, 35.8403, 35.8321, 35.7682, 35.7297, 35.7110,
         35.6348, 35.6074]]),
indices=tensor([[1467, 6502, 6555, 1882, 1575, 3176, 5377, 2799, 1673, 6328]]))
the federal reserve loves watching our manufacturers struggle with their exports to the benefit of other parts of the world has anyone looked at what almost all other countries are doing to take advantage of the good old usa our fed has been calling it wrong for too long 

the fake news is working overtime just reported that despite the tremendous success we are having with the economy amp all things else of the network news about me is negative fake why do we work so hard in working with the media when it is corrupt take away credentials 

delegation heading to china to begin talks on the massive trade deficit that has been created with our country very much like north korea this should have been fixed years ago not now same with other countries and nafta but it wil

In [ ]:

tweet_after[800]

In [ ]:
tweet_after[9335]

In [ ]:
tweet_after[4955]

In [ ]:
tweet_after[2434]